In [20]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.nn import LeakyReLU
from torch.utils.tensorboard import SummaryWriter

In [21]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 10

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

test_val_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

valset , testset = torch.utils.data.random_split(test_val_set, [0.4, 0.6])

valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [22]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    #    self.soft = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool(nn.LeakyReLU()(self.conv1(x)))
        x = self.pool(nn.LeakyReLU()(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = nn.LeakyReLU()(self.fc1(x))
        x = nn.LeakyReLU()(self.fc2(x))
        x = self.fc3(x)
    #    x = self.soft(x)
        return x
    
    
net = Net()

In [23]:
def val_model(epoch, best_loss):
    correct = 0
    total = 0
    accuracy = 0
    for i, data in enumerate(valloader, 0):

        inputs, labels = data

        outputs = net(inputs) #prediction

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loss = criterion(outputs, labels)
               
    accuracy = correct/total
    writer.add_scalar('Loss/validation', loss, (epoch+1))
    writer.add_scalar('Accuracy/validation', accuracy, (epoch+1))

    if loss < best_loss:
        best_loss = loss
        torch.save(net, "best_network.pt")

    return best_loss


In [24]:
def test_model():
    correct = 0
    total = 0
    accuracy = 0
    for i, data in enumerate(testloader, 0):

        inputs, labels = data

        outputs = best_net(inputs) #prediction

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loss = criterion(outputs, labels)
               
    accuracy = correct/total

    return accuracy

In [25]:
def train_model(epochs):
    best_loss = 100
    for epoch in range(epochs):
        correct = 0
        total = 0
        accuracy = 0
        for i, data in enumerate(trainloader, 0):

            inputs, labels = data

            outputs = net(inputs) #prediction

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            loss = criterion(outputs, labels)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()
        

        
        accuracy = correct/total
        writer.add_scalar('Loss/train', loss, (epoch+1))
        writer.add_scalar('Accuracy/train', accuracy, (epoch+1))

        best_loss = val_model(epoch,best_loss)


    


    return()

In [26]:

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.parameters(), lr=0.0001)

epochs = 30

writer = SummaryWriter()

train_model(epochs)

best_net = torch.load("best_network.pt")

best_accuracy = test_model()

print("The best network had the accuracy of %f" % (best_accuracy))


The best network had the accuracy of 0.291500
